In [ ]:
###############################################

In [ ]:
!pip install transformers
!pip install datasets
!pip install tiktoken
!pip install wandb
!pip install tqdm

In [52]:
import pandas as pd
data = pd.read_json('data/mtg_char/default-cards-20230313210730.json')

In [33]:
# peek at the data
print(data.shape)
data.head()

(80731, 84)


,object,id,oracle_id,multiverse_ids,mtgo_id,mtgo_foil_id,tcgplayer_id,cardmarket_id,name,lang,...,tcgplayer_etched_id,attraction_lights,color_indicator,variation_of,life_modifier,hand_modifier,printed_type_line,printed_text,content_warning,flavor_name
0,card,0000579f-7b35-4ed3-b44c-db2a538066fe,44623693-51d6-49ad-8cd7-140505caf02f,[109722],25527.0,25528.0,14240.0,13850.0,Fury Sliver,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,card,00006596-1166-4a79-8443-ca9f82e6db4e,8ae3562f-28b7-4462-96ed-be0cf7052ccc,[189637],34586.0,34587.0,33347.0,21851.0,Kor Outfitter,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,card,0000a54c-a511-4925-92dc-01b937f9afad,dc4e2134-f0c2-49aa-9ea3-ebf83af1445c,[],NaN,NaN,98659.0,NaN,Spirit,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,card,0000cd57-91fe-411f-b798-646e965eec37,9f0d82ae-38bf-45d8-8cda-982b6ead1d72,[435231],65170.0,65171.0,145764.0,301766.0,Siren Lookout,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,card,00012bd8-ed68-4978-a22d-f450c8a6e048,5aa12aff-db3c-4be5-822b-3afdf536b33e,[1278],NaN,NaN,1623.0,5664.0,Web,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
# filter out cards that are not in english
data = data[data['lang'] == 'en']
print(data.shape)
data.head()

(78312, 84)


,object,id,oracle_id,multiverse_ids,mtgo_id,mtgo_foil_id,tcgplayer_id,cardmarket_id,name,lang,...,tcgplayer_etched_id,attraction_lights,color_indicator,variation_of,life_modifier,hand_modifier,printed_type_line,printed_text,content_warning,flavor_name
0,card,0000579f-7b35-4ed3-b44c-db2a538066fe,44623693-51d6-49ad-8cd7-140505caf02f,[109722],25527.0,25528.0,14240.0,13850.0,Fury Sliver,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,card,00006596-1166-4a79-8443-ca9f82e6db4e,8ae3562f-28b7-4462-96ed-be0cf7052ccc,[189637],34586.0,34587.0,33347.0,21851.0,Kor Outfitter,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,card,0000a54c-a511-4925-92dc-01b937f9afad,dc4e2134-f0c2-49aa-9ea3-ebf83af1445c,[],NaN,NaN,98659.0,NaN,Spirit,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,card,0000cd57-91fe-411f-b798-646e965eec37,9f0d82ae-38bf-45d8-8cda-982b6ead1d72,[435231],65170.0,65171.0,145764.0,301766.0,Siren Lookout,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,card,00012bd8-ed68-4978-a22d-f450c8a6e048,5aa12aff-db3c-4be5-822b-3afdf536b33e,[1278],NaN,NaN,1623.0,5664.0,Web,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [54]:
# Keep only normal cards
data = data[data['layout'] == 'normal']

In [36]:
data.head()

,object,id,oracle_id,multiverse_ids,mtgo_id,mtgo_foil_id,tcgplayer_id,cardmarket_id,name,lang,...,tcgplayer_etched_id,attraction_lights,color_indicator,variation_of,life_modifier,hand_modifier,printed_type_line,printed_text,content_warning,flavor_name
0,card,0000579f-7b35-4ed3-b44c-db2a538066fe,44623693-51d6-49ad-8cd7-140505caf02f,[109722],25527.0,25528.0,14240.0,13850.0,Fury Sliver,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,card,00006596-1166-4a79-8443-ca9f82e6db4e,8ae3562f-28b7-4462-96ed-be0cf7052ccc,[189637],34586.0,34587.0,33347.0,21851.0,Kor Outfitter,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,card,0000cd57-91fe-411f-b798-646e965eec37,9f0d82ae-38bf-45d8-8cda-982b6ead1d72,[435231],65170.0,65171.0,145764.0,301766.0,Siren Lookout,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,card,00012bd8-ed68-4978-a22d-f450c8a6e048,5aa12aff-db3c-4be5-822b-3afdf536b33e,[1278],NaN,NaN,1623.0,5664.0,Web,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,card,0001f1ef-b957-4a55-b47f-14839cdbab6f,ef027846-be81-4959-a6b5-56bd01b1e68a,[472997],78170.0,NaN,198861.0,400134.0,Venerable Knight,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
# remove cards with duplicate names
data = data.drop_duplicates(subset=['name'])
print(data.shape)
data.head()

(25438, 84)


,object,id,oracle_id,multiverse_ids,mtgo_id,mtgo_foil_id,tcgplayer_id,cardmarket_id,name,lang,...,tcgplayer_etched_id,attraction_lights,color_indicator,variation_of,life_modifier,hand_modifier,printed_type_line,printed_text,content_warning,flavor_name
0,card,0000579f-7b35-4ed3-b44c-db2a538066fe,44623693-51d6-49ad-8cd7-140505caf02f,[109722],25527.0,25528.0,14240.0,13850.0,Fury Sliver,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,card,00006596-1166-4a79-8443-ca9f82e6db4e,8ae3562f-28b7-4462-96ed-be0cf7052ccc,[189637],34586.0,34587.0,33347.0,21851.0,Kor Outfitter,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,card,0000cd57-91fe-411f-b798-646e965eec37,9f0d82ae-38bf-45d8-8cda-982b6ead1d72,[435231],65170.0,65171.0,145764.0,301766.0,Siren Lookout,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,card,00012bd8-ed68-4978-a22d-f450c8a6e048,5aa12aff-db3c-4be5-822b-3afdf536b33e,[1278],NaN,NaN,1623.0,5664.0,Web,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,card,0001f1ef-b957-4a55-b47f-14839cdbab6f,ef027846-be81-4959-a6b5-56bd01b1e68a,[472997],78170.0,NaN,198861.0,400134.0,Venerable Knight,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [56]:
# now that we dont have any other languages, we can drop the lang column
data = data.drop(columns=['lang'])
print(data.shape)
data.head()

(25438, 83)


,object,id,oracle_id,multiverse_ids,mtgo_id,mtgo_foil_id,tcgplayer_id,cardmarket_id,name,released_at,...,tcgplayer_etched_id,attraction_lights,color_indicator,variation_of,life_modifier,hand_modifier,printed_type_line,printed_text,content_warning,flavor_name
0,card,0000579f-7b35-4ed3-b44c-db2a538066fe,44623693-51d6-49ad-8cd7-140505caf02f,[109722],25527.0,25528.0,14240.0,13850.0,Fury Sliver,2006-10-06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,card,00006596-1166-4a79-8443-ca9f82e6db4e,8ae3562f-28b7-4462-96ed-be0cf7052ccc,[189637],34586.0,34587.0,33347.0,21851.0,Kor Outfitter,2009-10-02,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,card,0000cd57-91fe-411f-b798-646e965eec37,9f0d82ae-38bf-45d8-8cda-982b6ead1d72,[435231],65170.0,65171.0,145764.0,301766.0,Siren Lookout,2017-09-29,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,card,00012bd8-ed68-4978-a22d-f450c8a6e048,5aa12aff-db3c-4be5-822b-3afdf536b33e,[1278],NaN,NaN,1623.0,5664.0,Web,1994-04-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,card,0001f1ef-b957-4a55-b47f-14839cdbab6f,ef027846-be81-4959-a6b5-56bd01b1e68a,[472997],78170.0,NaN,198861.0,400134.0,Venerable Knight,2019-10-04,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [57]:
data = data.drop(columns=['mtgo_id', 'mtgo_foil_id', 'arena_id', 'tcgplayer_id', 'cardmarket_id', 'legalities', 'games', 'reserved', 'foil', 'nonfoil', 'finishes', 'oversized', 'promo', 'reprint', 'variation', 'artist', 'artist_ids', 'illustration_id', 'border_color', 'booster', 'story_spotlight', 'edhrec_rank', 'penny_rank','prices', 'promo_types', 'arena_id', 'preview', 'security_stamp', 'tcgplayer_etched_id', 'variation_of','released_at', 'set_id','set', 'set_uri', 'set_search_uri', 'scryfall_set_uri', 'rulings_uri', 'prints_search_uri', 'card_back_id', 'frame', 'related_uris'])

In [58]:
data = data.drop(columns=['object', 'id', 'oracle_id', 'uri', 'scryfall_uri', 'layout', 'highres_image', 'image_status', 'printed_name', 'card_faces', 'attraction_lights', 'color_indicator', 'color_indicator', 'life_modifier', 'hand_modifier', 'printed_type_line', 'printed_text', 'content_warning', 'flavor_name'])

In [59]:
data = data.drop(columns=['full_art', 'textless', 'all_parts', 'produced_mana', 'watermark', 'loyalty', 'frame_effects', 'digital', 'cmc' ])

In [60]:
data = data.drop(columns=['keywords', 'collector_number', 'colors', 'color_identity'])

In [61]:
print(data.shape)
data.head()

(25438, 12)


,multiverse_ids,name,image_uris,mana_cost,type_line,oracle_text,power,toughness,set_name,set_type,rarity,flavor_text
0,[109722],Fury Sliver,{'small': 'https://cards.scryfall.io/small/fro...,{5}{R},Creature — Sliver,All Sliver creatures have double strike.,3,3,Time Spiral,expansion,uncommon,"""A rift opened, and our arrows were abruptly s..."
1,[189637],Kor Outfitter,{'small': 'https://cards.scryfall.io/small/fro...,{W}{W},Creature — Kor Soldier,"When Kor Outfitter enters the battlefield, you...",2,2,Zendikar,expansion,common,"""We take only what we need to survive. Believe..."
3,[435231],Siren Lookout,{'small': 'https://cards.scryfall.io/small/fro...,{2}{U},Creature — Siren Pirate,Flying\nWhen Siren Lookout enters the battlefi...,1,2,Ixalan,expansion,common,NaN
4,[1278],Web,{'small': 'https://cards.scryfall.io/small/fro...,{G},Enchantment — Aura,Enchant creature (Target a creature as you cas...,NaN,NaN,Revised Edition,core,rare,NaN
5,[472997],Venerable Knight,{'small': 'https://cards.scryfall.io/small/fro...,{W},Creature — Human Knight,"When Venerable Knight dies, put a +1/+1 counte...",2,1,Throne of Eldraine,expansion,uncommon,"""May this blade guide you on your great journe..."


In [62]:
data['oracle_text'] = data.apply(lambda x: x['oracle_text'].replace(x['name'], '<card_name>'), axis=1)


In [63]:
print(data.shape)
data.head()

(25438, 12)


,multiverse_ids,name,image_uris,mana_cost,type_line,oracle_text,power,toughness,set_name,set_type,rarity,flavor_text
0,[109722],Fury Sliver,{'small': 'https://cards.scryfall.io/small/fro...,{5}{R},Creature — Sliver,All Sliver creatures have double strike.,3,3,Time Spiral,expansion,uncommon,"""A rift opened, and our arrows were abruptly s..."
1,[189637],Kor Outfitter,{'small': 'https://cards.scryfall.io/small/fro...,{W}{W},Creature — Kor Soldier,"When <card_name> enters the battlefield, you m...",2,2,Zendikar,expansion,common,"""We take only what we need to survive. Believe..."
3,[435231],Siren Lookout,{'small': 'https://cards.scryfall.io/small/fro...,{2}{U},Creature — Siren Pirate,Flying\nWhen <card_name> enters the battlefiel...,1,2,Ixalan,expansion,common,NaN
4,[1278],Web,{'small': 'https://cards.scryfall.io/small/fro...,{G},Enchantment — Aura,Enchant creature (Target a creature as you cas...,NaN,NaN,Revised Edition,core,rare,NaN
5,[472997],Venerable Knight,{'small': 'https://cards.scryfall.io/small/fro...,{W},Creature — Human Knight,"When <card_name> dies, put a +1/+1 counter on ...",2,1,Throne of Eldraine,expansion,uncommon,"""May this blade guide you on your great journe..."


In [ ]:
# Build a string representation of each card to be saved out as a text file.
# Card data will be represented as a string with the following format:
# "<|endoftext|>{card_name} | ?{mana_cost} | {type_line} | {rarity} | ?{oracle_text} | ?{flavor_text} | ?{power} | ?{toughness}<|endoftext|>"
# The ? indicates that the field may be empty.

# start of text token
sot = "<|startoftext|>"
# end of text token
eot = "<|endoftext|>"
# pad token
pad = "<|pad|>"

card_data = []

for index, row in data.iterrows():
    card_data.append(f" {eot} {row['name']} | {row['mana_cost']} | {row['type_line']} | {row['rarity']} | {row['oracle_text']} | {row['flavor_text']} | {row['power']} | {row['toughness']} {eot} \n")

# sample the first 10 cards
card_data[:10]


In [ ]:
# Remove all new line characters from the data
card_data = [card.replace('\n', ' ') for card in card_data]

In [ ]:
# save the card data to a text file
with open('data/mtg_char/mtg_card_data.txt', 'w') as f:
    f.write(''.join(card_data))

In [ ]:
# write out the dataframe to a json file
data.to_json('data/mtg_char/mtg_card_data.json', orient='records', lines=True)

# Load the data into a dataset

In [ ]:
# Create a dataset from the text file
from datasets import load_dataset
dataset = load_dataset('text', data_files='data/mtg_char/mtg_card_data.json')


In [ ]:
# sample the first 10 cards
dataset['train'][:1]


In [ ]:
dataset['train'].shape

In [ ]:
# create a tokenizer

In [ ]:
from transformers import GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2', max_length=1024 , padding='max_length', pad_to_max_length=True, add_prefix_space=True, truncation=True, bos_token='<|endoftext|>', eos_token='<|endoftext|>', pad_token='<|pad|>', return_tensors='pt', mask_token='<|mask|>')


In [ ]:
# tokenize the dataset
tokenized_dataset = dataset.map(lambda examples: tokenizer(examples['text'], add_special_tokens=True ), batched=True)



In [ ]:
tokenized_dataset.shape

In [ ]:
# split the dataset into train and validation
tokenized_dataset = tokenized_dataset['train'].train_test_split(test_size=0.2)


In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [ ]:
# create a model
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained('gpt2-medium')


In [ ]:
# train the model
from transformers import Trainer, TrainingArguments
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=1,              # total # of training epochs
    per_device_train_batch_size=1,  # batch size per device during training
    per_device_eval_batch_size=1,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)


In [ ]:
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=tokenized_dataset['train'],         # training dataset
    eval_dataset=tokenized_dataset['test'],           # evaluation dataset
    data_collator=data_collator,         # data collator
)


In [ ]:
from transformers.integrations import WandbCallback

# disable the wandb logger
trainer.remove_callback(WandbCallback)

In [ ]:
trainer.train()

In [ ]:
# peek into the tokenized dataset
tokenized_dataset['train'][0]






In [ ]:
# Create a dataset from the text file
from datasets import load_dataset

dataset = load_dataset('text', data_files='data/mtg_char/mtg_card_data.json')

# sample the first 10 cards
dataset['train'][:1]

dataset['train'].shape
# create a tokenizer
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained('gpt2', max_length=1024, padding='max_length', pad_to_max_length=True, add_prefix_space=True, truncation=True, bos_token='', eos_token='', pad_token='<pad>', return_tensors='pt', mask_token='')


# tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['text'], add_special_tokens=True, padding='max_length', truncation=True, max_length=1024)


tokenized_dataset = dataset.map(tokenize_function, batched=True)

tokenized_dataset.shape
# split the dataset into train and validation
tokenized_dataset = tokenized_dataset['train'].train_test_split(test_size=0.2)

from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)
# create a model
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained('gpt2-medium')

# train the model
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',  # output directory
    num_train_epochs=1,  # total # of training epochs
    per_device_train_batch_size=1,  # batch size per device during training
    per_device_eval_batch_size=1,  # batch size for evaluation
    warmup_steps=500,  # number of warmup steps for learning rate scheduler
    weight_decay=0.01,  # strength of weight decay
    logging_dir='./logs',  # directory for storing logs
    logging_steps=10,
    device='cpu'
)

trainer = Trainer(
    model=model,  # the instantiated 🤗 Transformers model to be trained
    args=training_args,  # training arguments, defined above
    train_dataset=tokenized_dataset['train'],  # training dataset
    eval_dataset=tokenized_dataset['test'],  # evaluation dataset
    data_collator=data_collator,  # data collator
)

from transformers.integrations import WandbCallback

# disable the wandb logger
trainer.remove_callback(WandbCallback)

trainer.train()


In [ ]:
# Create a dataset from the text file
from datasets import load_dataset
dataset = load_dataset('text', data_files='data/mtg_char/mtg_card_data.json')

# create a tokenizer
from transformers import GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2', max_length=1024 , padding='max_length', pad_to_max_length=True, add_prefix_space=True, truncation=True, bos_token='', eos_token='', pad_token='<pad>', return_tensors='pt', mask_token='')

# tokenize the dataset
tokenized_dataset = dataset.map(lambda examples: tokenizer(examples['text'], add_special_tokens=True ), batched=True)

# split the dataset into train and validation
tokenized_dataset = tokenized_dataset['train'].train_test_split(test_size=0.2)

# create a model
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained('gpt2-medium')
model.resize_token_embeddings(len(tokenizer))

# train the model
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['test'],
    data_collator=data_collator
)

from transformers.integrations import WandbCallback

# disable the wandb logger
trainer.remove_callback(WandbCallback)

trainer.train()


In [ ]:
# Evalute the model
trainer.evaluate()


In [ ]:
# save the model
trainer.save_model('mtg_card_model')


In [ ]:
#sample the model output
from transformers import pipeline
unmasker = pipeline('fill-mask', model='mtg_card_model', tokenizer='gpt2')
unmasker('')




In [ ]:
#generate the model output
from transformers import pipeline
generator = pipeline('text-generation', model='mtg_card_model', tokenizer='gpt2', max_length=1024, do_sample=True, top_k=50, top_p=0.95, num_return_sequences=1, repetition_penalty=1.0, temperature=1.0, no_repeat_ngram_size=2, bad_words_ids=None, pad_token_id=50256, length_penalty=1.0, num_beams=1, early_stopping=False, use_cache=True, num_beam_groups=1, diversity_penalty=0.0, prefix_allowed_tokens_fn=None, output_attentions=None, output_hidden_states=None, output_scores=None, return_dict_in_generate=None, forced_bos_token_id=None, forced_eos_token_id=None, remove_invalid_values=None, return_dict=None)
generator('')


In [ ]:
# sample the first element of the dataset

dataset['train'][0]
dataset.shape

In [93]:
# create a function to get the base64 encoded art_crop image
import base64
import requests

def get_image_url(card):
    return card['image_uris']['art_crop']

#create a function to get the image from the url in base64 encoding
def get_image_base64(card):
    url = get_image_url(card)
    response = requests.get(url)
    return base64.b64encode(response.content).decode('utf-8')


In [94]:
# Try the function out on the first row of data
get_image_base64(data.iloc[0])

'...'

In [72]:
# Test the image classifier api

import requests

response = requests.post("https://persing-clip-interrogator.hf.space/run/clipit", json={
  "data": [
    "data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAAEAAAABCAYAAAAfFcSJAAAACklEQVR4nGMAAQAABQABDQottAAAAABJRU5ErkJggg==",
    "ViT-L (best for Stable Diffusion 1.*)",
    "best",
]}).json()

description = response["data"]


In [73]:
description

['a couple of giraffe standing on top of a lush green field, libra symbol, darkmode, drawn with a single line, glagolitic glyph, 100, in a dark room, by Sōami, nba logo, necromorph, full width, anamorpic lens, vertical, horizon centered, oled, gigapixel, top',
 {'visible': True, '__type__': 'update'},
 {'visible': True, '__type__': 'update'},
 {'visible': True, '__type__': 'update'}]

In [109]:
# create a function to get the description from the image

def get_description(image_base64):
    response = requests.post("https://persing-clip-interrogator.hf.space/run/clipit", json={
        "data": [
            "data:image/png;base64," + image_base64 ,
            "ViT-L (best for Stable Diffusion 1.*)",
            "fast",
        ]}).json()
    return response["data"]


In [96]:
# Test the function
get_description(get_image_base64(data.iloc[0]))

['a close up of a painting of a dragon, style of yoshitaka amano, yoshitaka amano character design, from magic the gathering, magic the gathering art style, the jabberwock, mtg art, from software game, by Yoshitaka Amano, stu_dts, ophanim has bird wings, eldrazi',
 {'visible': True, '__type__': 'update'},
 {'visible': True, '__type__': 'update'},
 {'visible': True, '__type__': 'update'}]

In [77]:
# add a column to the dataset with the art_description for the first 10 rows
data['art_description'] = data.iloc[:1].apply(lambda row: get_description(get_image_base64(row)), axis=1)


In [143]:
# time how long it takes to get the descriptions for the first 5 rows
import time
start = time.time()
data['art_description'] = data.iloc[:5].apply(lambda row: get_description(get_image_base64(row)), axis=1)
end = time.time()
print(end - start)



45.215770959854126


In [120]:
# get the size of the dataset
data.shape


(25438, 13)

In [82]:
# 135 seconds time the data size, then divide by 60*60 to get the hours
135*25438/(60*60)

953.925

In [98]:
# that was the time before I changed the api call to fast from best
12.5*25438/(60*60)

88.32638888888889

In [99]:
# thats still way too long, maybe reduce the resolution of the images


# Check the current size of the images
import requests
from io import BytesIO
from PIL import Image

url = get_image_url(data.iloc[0])
response = requests.get(url)
img = Image.open(BytesIO(response.content))
img.size


(616, 452)

In [100]:
# I wonder how acurate the image classifier is with the lower resolution images
# Let's try it with 256x256 images

# resize the image
img = img.resize((256, 256))
img.size

(256, 256)

In [101]:
# convert the image to base64
base64_image = base64.b64encode(BytesIO(response.content).getvalue()).decode('utf-8')

# get the description
description = get_description(base64_image)

TypeError: a bytes-like object is required, not 'Image'

In [88]:
description

["a close up of a painting of a dragon, konami concept art, necromorph, looking like a bird, from 2001, card game illustration, long boi, punpun onodera, by Kanō Tan'yū, darkpowerful elegant, angelic and unsettling, barren, bisley, persian, tabletop miniature",
 {'visible': True, '__type__': 'update'},
 {'visible': True, '__type__': 'update'},
 {'visible': True, '__type__': 'update'}]

In [102]:
# let's redefine the get_image_base64 function to resize the image to 256x256


# def get_image_base64(card):
#     url = get_image_url(card)
#     response = requests.get(url)
#     img = Image.open(BytesIO(response.content))
#     img = img.resize((256, 256))
#     img_byte_arr = BytesIO()
#     img.save(img_byte_arr, format='PNG')
#     img_byte_arr = img_byte_arr.getvalue()
#     return base64.b64encode(BytesIO(img_byte_arr).getvalue()).decode('utf-8')

In [103]:
# It looks like that still works, so let's try timing the first 5 rows again but this time resize the images to 256x256
start = time.time()
data['art_description'] = data.iloc[:5].apply(lambda row: get_description(get_image_base64(row)), axis=1)
end = time.time()
print(end - start)


13.386348724365234


In [108]:
# that actually took longer. I wonder if it's because the images are being resized. Let's try it again but this time resize the images before the api call

# here we will resize the images before the api call and store the base64 encoded image in the art_description column
data['art_description'] = data.iloc[:5].apply(lambda row: get_image_base64(row), axis=1)

# now we will get the description from the base64 encoded image and time it
start = time.time()
data['art_description'] = data.iloc[:5].apply(lambda row: get_description(row['art_description']), axis=1)
end = time.time()
print(end - start)

12.40333104133606


In [107]:
# That still took longer, I guess the model is already downsizing the images. So there is no point in downsizing the images before the api call

1678926153.998484

In [137]:
# let's see if all rows have a value for art_crop
data['image_uris'].isnull().sum()
# 0 rows have a null value for image_uris so now lets see if all of the image uris have an art_crop
data['image_uris'].apply(lambda x: x['art_crop'] if x is not None else None).isnull().sum()

# data['image_uris'][0]['art_crop']

0

In [145]:
# looks like we cant cut down on the number of descriptions we need to get maybe we can run this locally to avoid some costs


In [146]:
from transformers import pipeline

image_to_text = pipeline("image-to-text", model="nlpconnect/vit-gpt2-image-captioning")

image_to_text("https://ankur3107.github.io/assets/images/image-captioning-example.png")


/Users/nickpersing/PycharmProjects/nanoGPT/nanoGPT/venv/lib/python3.9/site-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(
/Users/nickpersing/PycharmProjects/nanoGPT/nanoGPT/venv/lib/python3.9/site-packages/transformers/generation/utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


[{'generated_text': 'a soccer game with a player jumping to catch the ball '}]

In [147]:
# let's try it on the first row of the dataset
image_to_text(get_image_url(data.iloc[0]))

[{'generated_text': 'a painting of a bird with a fish in its mouth '}]

In [148]:
# let's try it on the first 5 rows of the dataset and time it

start = time.time()
data['art_description'] = data.iloc[:5].apply(lambda row: image_to_text(get_image_url(row)), axis=1)
end = time.time()
print(end - start)


3.133573055267334
